# Gathering data

In [ ]:
#pip install yfinance, yesg

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import scipy.optimize as opt
import yesg

In [2]:
start_date = '2020-01-01'
end_date = '2023-12-31'

# get the return of the ETFs
quality = yf.Ticker('SPHQ').history(period='1d', start=start_date, end=end_date)['Close'].pct_change()
value = yf.Ticker('IVE').history(period='1d', start=start_date, end=end_date)['Close'].pct_change()
dividend = yf.Ticker('SPYD').history(period='1d', start=start_date, end=end_date)['Close'].pct_change()
low_vol = yf.Ticker('LOWV.L').history(period='1d', start=start_date, end=end_date)['Close'].pct_change()
momentum = yf.Ticker('SPMO').history(period='1d', start=start_date, end=end_date)['Close'].pct_change()

quality.index = pd.to_datetime(quality.index).strftime('%Y-%m-%d')
value.index = pd.to_datetime(value.index).strftime('%Y-%m-%d')
dividend.index = pd.to_datetime(dividend.index).strftime('%Y-%m-%d')
low_vol.index = pd.to_datetime(low_vol.index).strftime('%Y-%m-%d')
momentum.index = pd.to_datetime(momentum.index).strftime('%Y-%m-%d')


In [3]:
df = pd.concat([quality, value, dividend, low_vol, momentum], axis=1)
df.columns = ['Quality', 'Value', 'Dividend', 'Low Vol', 'Momentum']
df = df.dropna(how='any')

In [4]:
df

,Quality,Value,Dividend,Low Vol,Momentum
Date,,,,,
2020-01-03,-0.008130,-0.007121,-0.004342,0.000673,-0.003084
2020-01-06,0.001913,0.001619,0.003335,-0.001514,0.000738
2020-01-07,-0.003545,-0.003773,-0.002046,-0.000758,-0.005708
2020-01-08,0.005473,0.002628,0.001281,0.003372,0.005023
2020-01-09,0.007077,0.004471,0.001279,0.001260,0.010233
...,...,...,...,...,...
2023-12-21,0.009356,0.008913,0.009888,-0.005735,0.011688
2023-12-22,0.000927,0.004068,0.003350,0.003402,0.001848
2023-12-27,-0.000185,0.001900,0.001019,0.007297,0.003057


# Weights optimization of risk factors

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Function to calculate the cost of transactions
def transaction_costs(w, w0, TC):
    return np.sum(TC * np.abs(w - w0)**2)

# Modified objective function to include transaction costs
def objective(w, cov_mat, w0, TC, factor_TC):
    # Original variance-based function
    s = 0
    for i in range(len(w)):
        for j in range(len(w)):
            s += (w[i] * (cov_mat @ w)[i] - w[j] * (cov_mat @ w)[j])**2
    # Add transaction costs as a penalty term
    return s + factor_TC * transaction_costs(w, w0, TC)

# Example data
cov_mat = df.cov().values  # Covariance matrix of ETF returns
w0 = np.array([0.3, 0.2, 0.1, 0.2, 0.2])  # Initial weights (current portfolio)
TC = np.array([0.0005, 0.0005, 0.0005, 0.0005, 0.0005])  # Transaction costs (5 bps)
factor_TC = .0001  # Scaling factor for transaction costs (tune this as needed)

# Constraints: weights sum to 1 and are long-only
constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},  # Sum of weights = 1
               {'type': 'ineq', 'fun': lambda w: w}]            # Weights must be positive (long only)

# Optimize
result = minimize(objective, w0, args=(cov_mat, w0, TC, factor_TC), constraints=constraints, tol=1e-20)
w_opt = result.x  # Optimized weights
print("Optimal weights:", w_opt)

Optimal weights: [0.19913861 0.18458651 0.15118808 0.27957819 0.1855086 ]


Check that each marginal risk contribution is the same

In [11]:
sigma = np.sqrt(w_opt.T@cov_mat@w_opt)

for i in range(5):
    s= (w_opt @ cov_mat[i]) / sigma
    print("Marginal contribution of factor {} : {}".format(i,w_opt[i]*s))

Marginal contribution of factor 0 : 0.0026936287424913566
Marginal contribution of factor 1 : 0.002494598092371917
Marginal contribution of factor 2 : 0.00235124012271799
Marginal contribution of factor 3 : 0.0022059798723483713
Marginal contribution of factor 4 : 0.0024799045773492147


# From ETFs weights to stocks weights

In [28]:
import pandas as pd
import requests
from io import BytesIO, StringIO

def download_data(url):
    """
    Télécharge et retourne les données d'une URL, en gérant les fichiers Excel et CSV de manière appropriée.
    """
    if url.endswith('.xls') or url.endswith('.xlsx'):
        skip_rows = 4
        response = requests.get(url)
        response.raise_for_status()
        data = pd.read_excel(BytesIO(response.content), skiprows=skip_rows)
    else:
        skip_rows = 0
        response = requests.get(url)
        response.raise_for_status()
        csv_data = response.content.decode('utf-8')
        try:
            data = pd.read_csv(StringIO(csv_data), skiprows=skip_rows)
        except pd.errors.ParserError:
            try:
                data = pd.read_csv(StringIO(csv_data), delimiter=';', skiprows=skip_rows)
            except pd.errors.ParserError:
                data = pd.read_csv(StringIO(csv_data), skiprows=9 + skip_rows)

    if not any('weight' in col.lower() for col in data.columns):
        data = pd.read_csv(StringIO(csv_data), skiprows=9 + skip_rows)
    return data

def normalize_ticker(ticker):
    """
    Normalise le ticker en supprimant les espaces et les caractères inutiles.
    """
    if pd.isna(ticker):  # Vérifier si ticker est NaN
        return ''
    return ticker.replace('/', '').strip().upper()

def calculate_total_weights_per_etf(url, etf_weight):
    """
    Calcule le poids total pour chaque action d'un fichier CSV ou Excel.
    """
    data = download_data(url)
    weight_column = next((col for col in data.columns if 'weight' in col.lower()), None)
    if weight_column is None:
        print("Colonne 'Weight' ou 'Weight (%)' introuvable dans le fichier.")
        return None

    holding_ticker_column = next((col for col in data.columns if 'holding ticker' in col.lower()), None)
    ticker_column = next((col for col in data.columns if 'ticker' in col.lower()), None)
    if holding_ticker_column:
        ticker_column = holding_ticker_column
    elif not ticker_column:
        print("Aucune colonne 'Ticker' ou 'Holding Ticker' introuvable dans le fichier.")
        return None

    data['Weighted_Weight'] = data[weight_column] * etf_weight

    if 'Name' in data.columns:
        data = data[~data['Name'].isin(["US DOLLAR", "Cash/Receivables/Payables", "S+P500 EMINI FUT  DEC24", "CASH COLLATERAL MSFUT USD", "USD CASH", "SSI US GOV MONEY MARKET CLASS"])]

    # Normaliser les tickers
    data[ticker_column] = data[ticker_column].apply(normalize_ticker)
    
    total_weights = data.groupby(ticker_column)[['Name', 'Weighted_Weight']].sum().reset_index()
    total_weights.columns = ['Ticker', 'Name', 'Total_Weight']
    total_weights = total_weights[total_weights['Total_Weight'] > 0]
    return total_weights

def calculate_total_weights(weights):
    # Téléchargement et traitement des données pour chaque URL
    urls = [
        "https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=SPHQ",
        "https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=SPLV",
        "https://www.invesco.com/us/financial-products/etfs/holdings/main/holdings/0?audienceType=Investor&action=download&ticker=SPMO",
        "https://www.ishares.com/us/products/239728/ishares-sp-500-value-etf/1467271812596.ajax?fileType=csv&fileName=IVE_holdings&dataType=fund",
        "https://www.ssga.com/us/en/intermediary/library-content/products/fund-data/etfs/us/holdings-daily-us-en-spyd.xlsx"
    ]

    final_data = pd.DataFrame()

    # Appliquer chaque URL et poids d'ETF
    for url, weight in zip(urls, weights):
        etf_data = calculate_total_weights_per_etf(url, weight)
        if etf_data is not None:
            final_data = pd.concat([final_data, etf_data])

    # Étape finale : Regrouper par ticker exact et sommer les poids
    final_data = final_data.groupby('Ticker', as_index=False)['Total_Weight'].sum()
    final_data['Total_Weight'] /= final_data['Total_Weight'].sum()

    # Trier les données en ordre décroissant de poids
    #final_data.sort_values(by='Total_Weight', ascending=False, inplace=True)

    # Vérification des doublons
    duplicate_tickers = final_data[final_data.duplicated(subset='Ticker', keep=False)]
    if not duplicate_tickers.empty:
        print("Doublons trouvés dans les tickers :")
        print(duplicate_tickers)
    #else:
    #    print("Aucun doublon trouvé dans les tickers.")

    # Affichage du résultat final
    #print(final_data.to_string(index=False))

    # Enregistrer dans un fichier CSV
    #final_data.to_csv("poids_actions_etf.csv", index=False)
    #print("Les données complètes ont été enregistrées dans 'poids_actions_etf.csv'.")
    
    return final_data

#Stocks with several classes: fox and news corp

In [36]:
stock_weights = calculate_total_weights(w_opt)
stock_weights.sort_values(by='Total_Weight', ascending=False)

,Ticker,Total_Weight
66,BRKB,0.023806
248,JPM,0.018787
44,AVGO,0.018738
104,COST,0.018523
246,JNJ,0.017155
...,...,...
121,DAY,0.000060
183,FOX,0.000060
31,AMTM,0.000060
140,DVA,0.000060


# Constraints

- We do not take into account the worst stocks in terms of ESG scores
- Sector constraints (the s&p is already concentrated so it might be interesting to allow a difference with the sector wiehgts in the original s&p 500)
- Concentration limits (a limit for each stock)
- Liquidity constraints (use only stocks with volume higher than a threshold): not so interesting because all s&p 500 stocks are liquid
- no short allowed
- tracking error: determined with backtest

### ESG constraints

In [35]:
# get the s&p 500 tickers with their name, sector and sub-industry
df_sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0][['Symbol', 'Security', 'GICS Sector', 'GICS Sub-Industry']]
df_sp500 = df_sp500.set_index('Symbol') # set the index to be the symbol

df_sp500.drop(index='GOOG', inplace=True) # drop the GOOG row because there is already a GOOGL row

df_esg = df_sp500.copy()
for ticker in df_sp500.index:
    try:
        df_esg.loc[ticker, "ESG Score"] = yesg.get_historic_esg(ticker).iloc[-1,0]
    except AttributeError:
        pass

In [36]:
#list of tickers with missing ESG score
df_esg[df_esg.isna().values].index
#Should we keep them or drop them?
#df_esg.dropna(axis=0, inplace=True)

Index([], dtype='object', name='Symbol')

In [37]:
#The best ESG score is 0
#drop the worst decile of the ESG scores (keep the top 90%)
quantile_threshold = 0.9
df_esg = df_esg[df_esg['ESG Score'] < df_esg['ESG Score'].quantile(quantile_threshold)]

#drop all the stocks that have an ESG score above the threshold
#threshold = 30
#df_esg = df_esg[df_esg['ESG Score'] < threshold]

In [140]:
def esg_weightening(df_weights):
    """ Input: a dataframe with the weights of the stocks before the ESG constraints
        Output: a dataframe with the weights of the stocks after the ESG constraints"""
    df_weights = df_weights.loc[df_esg.index]
    df_weights['Weights'] = df_weights['Weights'] / df_weights['Weights'].sum()
    return df_weights

#This is just for a test, it will be changed when we have the weights for each stock
equal_weights = df_sp500.copy()
equal_weights['Weights'] = 1/len(equal_weights)

df_weights_after_esg = esg_weightening(equal_weights)
df_weights_after_esg

,Security,GICS Sector,GICS Sub-Industry,Weights
Symbol,,,,
AOS,A. O. Smith,Industrials,Building Products,0.055556
ABT,Abbott Laboratories,Health Care,Health Care Equipment,0.055556
ACN,Accenture,Information Technology,IT Consulting & Other Services,0.055556
ADBE,Adobe Inc.,Information Technology,Application Software,0.055556
AMD,Advanced Micro Devices,Information Technology,Semiconductors,0.055556
AES,AES Corporation,Utilities,Independent Power Producers & Energy Traders,0.055556
AFL,Aflac,Financials,Life & Health Insurance,0.055556
A,Agilent Technologies,Health Care,Life Sciences Tools & Services,0.055556
APD,Air Products,Materials,Industrial Gases,0.055556


### Sector constraints

In [145]:
sector_threshold = 0.17
#We calculate the weights of the stocks in each sector
df_weights_by_sector = df_weights_after_esg[['GICS Sector', 'Weights']].groupby('GICS Sector').sum()
#We create a list of the sectors that have a weight above the threshold
sectors_above_threshold = df_weights_by_sector.loc[df_weights_by_sector['Weights'] > sector_threshold].index
while len(sectors_above_threshold) > 0:
    for sector in sectors_above_threshold:
        #for each stock in the sector, we apply a factor such that the sum of all stocks in this sector is equal to the threshold
        df_sector = df_weights_after_esg[df_weights_after_esg['GICS Sector'] == sector]
        factor = sector_threshold / df_weights_by_sector.loc[sector, 'Weights']
        df_weights_after_esg.loc[df_sector.index, 'Weights'] = df_sector['Weights'] * factor
    
    #we increase the weights of the stocks in the other sectors to keep the sum of the weights equal to 1
    df_weights_after_esg.loc[~df_weights_after_esg['GICS Sector'].isin(sectors_above_threshold), "Weights"] /= df_weights_after_esg.loc[~df_weights_after_esg['GICS Sector'].isin(sectors_above_threshold), "Weights"].sum() / (1-sector_threshold*len(sectors_above_threshold))
    
    #Some sectors may now have a weight above the threshold so we repeat the process
    df_weights_by_sector = df_weights_after_esg[['GICS Sector', 'Weights']].groupby('GICS Sector').sum()
    sectors_above_threshold = df_weights_by_sector.loc[df_weights_by_sector['Weights'] > sector_threshold].index

df_weights_after_esg

,Security,GICS Sector,GICS Sub-Industry,Weights
Symbol,,,,
AOS,A. O. Smith,Industrials,Building Products,0.060000
ABT,Abbott Laboratories,Health Care,Health Care Equipment,0.056667
ACN,Accenture,Information Technology,IT Consulting & Other Services,0.042500
ADBE,Adobe Inc.,Information Technology,Application Software,0.042500
AMD,Advanced Micro Devices,Information Technology,Semiconductors,0.042500
AES,AES Corporation,Utilities,Independent Power Producers & Energy Traders,0.060000
AFL,Aflac,Financials,Life & Health Insurance,0.060000
A,Agilent Technologies,Health Care,Life Sciences Tools & Services,0.056667
APD,Air Products,Materials,Industrial Gases,0.060000


In [146]:
#Check that none of the sectors have a weight above the threshold
df_weights_after_esg[['GICS Sector', 'Weights']].groupby('GICS Sector').sum()

,Weights
GICS Sector,
Communication Services,0.06
Consumer Discretionary,0.06
Financials,0.12
Health Care,0.17
Industrials,0.12
Information Technology,0.17
Materials,0.12
Real Estate,0.06
Utilities,0.12


In [147]:
#Check that the sum of the weights is equal to 1
df_weights_after_esg[['GICS Sector', 'Weights']].groupby('GICS Sector').sum().sum()

Weights    1.0
dtype: float64

### Stock constraints

In [148]:
# This is just for a test, it will disappear when we have the weights for each stock
df_weights_after_esg.iloc[-2,-1]=.08
df_weights_after_esg.iloc[-1,-1]=.08
df_weights_after_esg

,Security,GICS Sector,GICS Sub-Industry,Weights
Symbol,,,,
AOS,A. O. Smith,Industrials,Building Products,0.060000
ABT,Abbott Laboratories,Health Care,Health Care Equipment,0.056667
ACN,Accenture,Information Technology,IT Consulting & Other Services,0.042500
ADBE,Adobe Inc.,Information Technology,Application Software,0.042500
AMD,Advanced Micro Devices,Information Technology,Semiconductors,0.042500
AES,AES Corporation,Utilities,Independent Power Producers & Energy Traders,0.060000
AFL,Aflac,Financials,Life & Health Insurance,0.060000
A,Agilent Technologies,Health Care,Life Sciences Tools & Services,0.056667
APD,Air Products,Materials,Industrial Gases,0.060000


In [150]:

stock_threshold = 0.07
#We create a list of the stocks that have a weight above the threshold
stocks_above_threshold = df_weights_after_esg.loc[df_weights_after_esg['Weights'] > stock_threshold].index
while len(stocks_above_threshold) > 0:
    #we set the weights of the stocks above the threshold to the threshold
    df_weights_after_esg.loc[stocks_above_threshold, 'Weights'] = stock_threshold
    #we increase (proportionally) the weights of the other stocks to keep the sum of the weights equal to 1
    df_weights_after_esg.loc[~df_weights_after_esg.index.isin(stocks_above_threshold), "Weights"] /= df_weights_after_esg.loc[~df_weights_after_esg.index.isin(stocks_above_threshold), "Weights"].sum() / (1-stock_threshold*len(stocks_above_threshold))
    #Some stocks may now have a weight above the threshold so we repeat the process
    stocks_above_threshold = df_weights_after_esg.loc[df_weights_after_esg['Weights'] > stock_threshold].index

df_weights_after_esg

,Security,GICS Sector,GICS Sub-Industry,Weights
Symbol,,,,
AOS,A. O. Smith,Industrials,Building Products,0.058636
ABT,Abbott Laboratories,Health Care,Health Care Equipment,0.055379
ACN,Accenture,Information Technology,IT Consulting & Other Services,0.041534
ADBE,Adobe Inc.,Information Technology,Application Software,0.041534
AMD,Advanced Micro Devices,Information Technology,Semiconductors,0.041534
AES,AES Corporation,Utilities,Independent Power Producers & Energy Traders,0.058636
AFL,Aflac,Financials,Life & Health Insurance,0.058636
A,Agilent Technologies,Health Care,Life Sciences Tools & Services,0.055379
APD,Air Products,Materials,Industrial Gases,0.058636


In [151]:
#Check that the sum of the weights is equal to 1
df_weights_after_esg[['GICS Sector', 'Weights']].groupby('GICS Sector').sum().sum()

Weights    1.0
dtype: float64

### Liquidity constraints
We can limit the universe by only taking into account stocks with a volume higher than a threshold but I'm not sure it's necessary because all s&p 500 stocks are liquid.

### Tracking error constraints
Apart from using weights closer to the original s&p 500 weights, I don't really see how to reduce the tracking error. 

# References
### Implementations of equal risk contribution
- https://github.com/matthewgilbert/erc/blob/master/erc/erc.py
- https://github.com/mirca/riskparity.py (not used)
- https://thequantmba.wordpress.com/2016/12/14/risk-parityrisk-budgeting-portfolio-in-python/

### Papers
- [Paper of Maillard, Roncalli and Teiletche](http://thierry-roncalli.com/download/erc.pdf)
- [Slides of Maillard, Roncalli and Teiletche](http://www.thierry-roncalli.com/download/erc-slides.pdf)
- [Master's thesis of David Stefanovits](https://ethz.ch/content/dam/ethz/special-interest/math/risklab-dam/documents/walter-saxer-preis/ma-stefanovits.pdf)
